In [0]:
import math
from pyspark.sql.functions import *

In [0]:
# Reeading the dataset , splitting it into rows
friends_data = sc.textFile("/FileStore/tables/data.txt")

In [0]:
friends_data_split = friends_data.map(lambda x: x.split('\t'))

In [0]:
#friends_data_split.collect()

In [0]:
friends_rdd = friends_data_split.flatMap(lambda x: ((x[0],i) for i in x[1].split(',')) )
#friends_rdd.collect()

In [0]:
# EXPANDING THE FRIENDS OF EACH USER ID WITH THEIR FRIENDS

In [0]:
friends_inverted = friends_rdd.map( lambda x: (x[1],x[0]) )  
#friends_inverted.collect()

In [0]:
friends_joined = friends_inverted.join(friends_data_split)
#friends_joined.collect()

In [0]:
friends_reverted = friends_joined.map(lambda x:(x[1][0],(x[0],x[1][1])))
#friends_reverted.sortByKey().collect()

In [0]:
friends_reverted2=friends_reverted.map(lambda x:(x[0],x[1][1]))
#friends_reverted2.sortByKey().collect()

In [0]:
friends_union = friends_reverted2.reduceByKey(lambda x,y: (x +","+ y))
#friends_union.sortByKey().collect()

In [0]:
friends_suggestion_flatmap = friends_union.flatMap(lambda x:((x[0],i) for i in x[1].split(',')))
#friends_suggestion_flatmap.sortByKey().collect()

In [0]:
#friends_inverted.sortByKey().collect()

In [0]:
friends_suggestions_filtered = friends_suggestion_flatmap.subtract(friends_inverted)
#friends_suggestions_filtered.sortByKey().collect()

In [0]:
friends_suggestions_filtered_count = friends_suggestions_filtered.map(lambda x:(x[0],(x[1],1))) 

In [0]:
#friends_suggestions_filtered_count.sortByKey().collect()

In [0]:
friends_suggestions_filtered_count2 = friends_suggestions_filtered_count.filter(lambda x: x[1][0]!=x[0])
#friends_suggestions_filtered_count2.sortByKey().collect()

In [0]:
friends_mapped = friends_suggestions_filtered_count2.map(lambda x:((x[0]+"_"+x[1][0]),x[1][1]))
#friends_mapped.sortByKey().collect()

In [0]:
friends_reduced = friends_mapped.reduceByKey(lambda x,y:x+y)
#friends_reduced.sortByKey().collect()

In [0]:
friends_reduced2 = friends_reduced.map(lambda x: (x[0][0:x[0].index("_")],(x[0][x[0].index("_")+1:len(x[0])], x[1])))
#friends_reduced2.sortByKey().collect()

In [0]:
friends_reduced3 = friends_reduced2.sortBy(lambda x: -x[1][1]).sortByKey()


In [0]:
#friends_reduced3.collect()

In [0]:
friends_reduced4 = friends_reduced3.map(lambda x:(x[0],(x[1][0]))).reduceByKey(lambda x,y:((x+","+y))).sortByKey()
#friends_reduced4.collect()

In [0]:
# FINAL OUTPUT ==> TOP 10 FRIEND RECOMMENDATIONS FOR EACH USER ID

In [0]:
friends_final = friends_reduced4.map(lambda x:(x[0],(x[1].split(",")[0:10])))
friends_final.collect()